# Calculating Values in a Query

## Math Operators
- addition +
- subtraction -
- multiplication *
- division /
- modulus %
- Work differently on different data types

## Calculating Difference

In [1]:
from sqlalchemy import create_engine,Table, MetaData, select, desc, func

engine = create_engine('sqlite:///census.sqlite')
connection = engine.connect()

metadata = MetaData()

census = Table('census', metadata, autoload=True, autoload_with=engine)

In [2]:
stmt = select([census.columns.age,
               (census.columns.pop2008-census.columns.pop2000).label('pop_change')])


stmt = stmt.group_by(census.columns.age)  # group by age
stmt = stmt.order_by(desc('pop_change'))  # order by  new label
stmt = stmt.limit(5)

results = connection.execute(stmt).fetchall()

print(results)

[(61, 52672), (85, 51901), (54, 50808), (58, 45575), (60, 44915)]


## Case Statement
- Used to treat data differently based on a condition
- Accepts a list of conditions to match and a column
to return if the condition matches
- The list of conditions ends with an else clause to
determine what to do when a record doesn’t match
any prior conditions

## Case Example

In [3]:
from sqlalchemy import case

stmt = select([
    func.sum(
        case([
            (census.columns.state == 'New York', # check the col name is NewYork
             census.columns.pop2008)  # if that is the case, return the value of the pop2008 column
        ], else_=0))])   # return 0 where the state is not NewYork

results = connection.execute(stmt).fetchall()

print(results)

[(19465159,)]


## Cast Statement
- Converts data to another type
- Useful for converting
- integers to floats for division
- strings to dates and times
- Accepts a column or expression and the target Type

## Percentage Example

- What percentage of the total population lived in New York in 2008

In [5]:

from sqlalchemy import case, cast, Float

# sum all the rows of pop2008 where the state is New York, then divide it by the total 2008 population

stmt = select([
    (func.sum(
        case([
            (census.columns.state == 'New York', # s
             census.columns.pop2008)
        ], else_=0)) /  # divide
     cast(func.sum(census.columns.pop2008),  # total p0p2008, cast as a float
          Float) * 100).label('ny_percent')])  # multiply by 100 & create a new label called ny_percent

results = connection.execute(stmt).fetchall()

print(results)

[(6.426761976501632,)]


---
# Let’s practice!

```python
# Import create_engine function
from sqlalchemy import create_engine

# Create an engine to the census database
engine = create_engine('mysql+pymysql://'+
                        'student:datacamp'+
                        '@courses.csrrinzqubik.us-east-1.rds.amazonaws.com:3306/' 
                        + 'census'  )

# Print the table names
print(engine.table_names())
```

In [9]:
# Build query to return state names by population difference from 2008 to 2000: stmt
stmt = select([census.columns.state, (census.columns.pop2008-census.columns.pop2000).label('pop_change')])

# Append group by for the state: stmt
stmt = stmt.group_by(census.columns.state)

# Append order by for pop_change descendingly: stmt
stmt = stmt.order_by(desc('pop_change'))

# Return only 5 results: stmt
stmt = stmt.limit(5)

# Use connection to execute the statement and fetch all results
results = connection.execute(stmt).fetchall()

# Print the state and population change for each record
for result in results:
    print('{}:{}'.format(result.state, result.pop_change))

California:105705
Florida:100984
Texas:51901
New York:47098
Pennsylvania:42387


In [10]:
# import case, cast and Float from sqlalchemy
from sqlalchemy import case, cast, Float

# Build an expression to calculate female population in 2000
female_pop2000 = func.sum(
    case([
        (census.columns.sex == 'F', census.columns.pop2000)
    ], else_=0))

# Cast an expression to calculate total population in 2000 to Float
total_pop2000 = cast(func.sum(census.columns.pop2000), Float)

# Build a query to calculate the percentage of females in 2000: stmt
stmt = select([female_pop2000 / total_pop2000* 100])

# Execute the query and store the scalar result: percent_female
percent_female = connection.execute(stmt).scalar()

# Print the percentage
print(percent_female)

51.09467432293413


# SQL Relationships

## Relationships
- Allow us to avoid duplicate data
- Make it easy to change things in one place
- Useful to break out information from a table we
don’t need very often

## Automatic Joins

- build statement with the columns that we desire
- execute the query , print results.

- sqlalchemy correctly adds the right JOIN clause bc its predefined in the database

```python
In [1]: stmt = select([census.columns.pop2008,
...: state_fact.columns.abbreviation])
In [2]: results = connection.execute(stmt).fetchall()
In [3]: print(results)
Out[3]: [(95012, u'IL'),
(95012, u'NJ'),
(95012, u'ND'),
(95012, u'OR'),
(95012, u'DC'),
(95012, u’WI'),
 ```

## Join
- Accepts a Table and an optional expression that explains
how the two tables are related
- The expression is not needed if the relationship is
predefined and available via reflection
- Comes immediately a"er the select() clause and prior
to any where(), order_by or group_by() clauses

## Select_from
- Used to replace the default, derived FROM clause with a join
- Wraps the join() clause

## Select_from Example

- total population in 2000 that was withing the state circuit courtn  restriction

```python
In [1]: stmt = select([func.sum(census.columns.pop2000)])  # get column from pop200 talbe 
In [2]: stmt = stmt.select_from(census.join(state_fact))  # append the select_from to include the census table, join with the state_fact table
In [3]: stmt = stmt.where(state_fact.columns.circuit_court == '10')  # use where clause to only show whre value of columns.circuit_court is 10
In [4]: result = connection.execute(stmt).scalar()
In [5]: print(result)
Out[5]: 14945252
              ```

## Joining Tables without Predefined Relationship
- Join accepts a Table and an optional expression that explains
how the two tables are related
- Will only join on data that match between the two columns
- Avoid joining on columns of different types

## Select_from Example

- total population in 2008 that belongs to the 'East South Central' division of the census

- population and division live in diffrent tables

```python
 # select the sum of the pop200
In [1]: stmt = select([func.sum(census.columns.pop2000)])  # select
In [2]: stmt = stmt.select_from(
...: census.join(state_fact, census.columns.state
...: == state_fact.columns.name))
In [3]: stmt = stmt.where(
...: state_fact.columns.census_division_name ==
...: 'East South Central')
    
    
    
In [4]: result = connection.execute(stmt).scalar()
In [5]: print(result)
Out[5]: 16982311
    ```

---
# Let’s practice!


## Automatic Joins with an Established Relationship

```python
# Build a statement to join census and state_fact tables: stmt
stmt = select([census.columns.pop2000, state_fact.columns.abbreviation])

# Execute the statement and get the first result: result
result = connection.execute(stmt).first()

# Loop over the keys in the result object and print the key and value
for key in result.keys():
    print(key, getattr(result, key))
    ```


### Joins
```python
# Build a statement to select the census and state_fact tables: stmt
stmt = select([census, state_fact])

# Add a select_from clause that wraps a join for the census and state_fact
# tables where the census state column and state_fact name column match
stmt = stmt.select_from(
    census.join(state_fact, census.columns.state == state_fact.columns.name))

# Execute the statement and get the first result: result
result = connection.execute(stmt).first()

# Loop over the keys in the result object and print the key and value
for key in result.keys():
    print(key, getattr(result, key))
    ```


More Practice with Joins

```python
# Build a statement to select the state, sum of 2008 population and census
# division name: stmt
stmt = select([
    census.columns.state,
    func.sum(census.columns.pop2008),
    state_fact.columns.census_division_name
])

# Append select_from to join the census and state_fact tables by the census state and state_fact name columns
stmt = stmt.select_from(
    census.join(state_fact, census.columns.state == state_fact.columns.name)
)

# Append a group by for the state_fact name column
stmt = stmt.group_by(state_fact.columns.name)

# Execute the statement and get the results: results
results = connection.execute(stmt).fetchall()

# Loop over the the results object and print each record.
for record in results:
    print(record)

    ```

# Working with Hierarchical Tables

## Hierarchical Tables
- Contain a relationship with themselves
- Commonly found in:
    - Organizational
    - Geographic
    - Network
    - Graph

## Hierarchical Tables - alias()
- Requires a way to view the table via multiple names
- Creates a unique reference that we can use

## Querying Hierarchical Data

- get a list of managers amd the employees they report to them

```python
In [1]: managers = employees.alias()
    
In [2]: stmt = select(
...: [managers.columns.name.label('manager'),
...: employees.columns.name.label('employee')])
    
    
In [3]: stmt = stmt.select_from(employees.join(
...: managers, managers.columns.id ==
...: employees.columns.manager)
                                
In [4]: stmt = stmt.order_by(managers.columns.name)
                                 
In [5]: print(connection.execute(stmt).fetchall())
Out[5]: [(u'FILLMORE', u'GRANT'),
(u'FILLMORE', u'ADAMS'),
(u'HARDING', u'TAFT'), ...
         ```

## Group_by and Func
- It’s important to target group_by() at the right alias
- Be careful with what you perform functions on
- If you don’t find yourself using both the alias and the table
name for a query, don’t create the alias at all

## Querying Hierarchical Data

```python
In [1]: managers = employees.alias()
In [2]: stmt = select([managers.columns.name,
...: func.sum(employees.columns.sal)])
In [3]: stmt = stmt.select_from(employees.join(
...: managers, managers.columns.id ==
...: employees.columns.manager)
In [4]: stmt = stmt.group_by(managers.columns.name)
In [5]: print(connection.execute(stmt).fetchall())
Out[5]: [(u'FILLMORE', Decimal('96000.00')),
(u'GARFIELD', Decimal('83500.00')),
(u'HARDING', Decimal('52000.00')),
(u'JACKSON', Decimal('197000.00'))]
                                ```

---
# Let’s practice!

# Handling Large ResultSets

## Dealing with Large ResultSets
- fetchmany() lets us specify how many rows we want to
act upon
- We can loop over fetchmany()
- It returns an empty list when there are no more records
- We have to close the ResultProxy a"erwards

## Fetching Many Rows

```python
In [1]: while more_results:
...: partial_results = results_proxy.fetchmany(50)
...: if partial_results == []:
...: more_results = False
...: for row in partial_results:
...: state_count[row.state] += 1
In [2]: results_proxy.close()
    ```

---
# Let’s practice!